In [4]:
from dgfip.read import get_iris, get_structures
import numpy as np

In [2]:
structures = get_structures()
centres = structures[structures['TYPE DE SERVICE']=='Centre de Finances publiques']

In [7]:
import geopandas as gpd
# fond de carte des départements
dep = gpd.read_file('https://github.com/gregoiredavid/france-geojson/raw/master/departements-version-simplifiee.geojson', crs='4236')

In [5]:
x= np.array(centres.to_crs(crs='2154').geometry.x)
y= np.array(centres.to_crs(crs='2154').geometry.y)
points = np.stack((x,y), axis=-1)

In [8]:
from shapely.geometry import box
from libpysal.cg.voronoi  import voronoi, voronoi_frames
points_u, index = np.unique(points, axis=0, return_index=True)
region_df, point_df = voronoi_frames(points_u, clip = box(*dep.to_crs('2154').total_bounds))

In [58]:
areas = region_df.set_crs("2154").clip(dep.to_crs('2154')).area/10**6

In [59]:
region_df['area'] = areas

In [17]:
centres_u = centres.iloc[index]

In [22]:
iris = get_iris(compute_distance=False)

In [38]:
region_df['id'] = region_df.index

In [60]:
inter = region_df.set_crs('2154').sjoin(iris)

In [61]:
gr = inter.groupby('id').sum()

In [62]:
geo = gpd.GeoDataFrame(gr, geometry= region_df.geometry)

In [63]:
geo

,area,index_right,MODIF_IRIS,P17_POP,P17_POP0002,P17_POP0305,P17_POP0610,P17_POP1117,P17_POP1824,P17_POP2539,...,C17_F15P_CS5,C17_F15P_CS6,C17_F15P_CS7,C17_F15P_CS8,P17_POP_FR,P17_POP_ETR,P17_POP_IMM,P17_PMEN,P17_PHORMEN,geometry
id,,,,,,,,,,,,,,,,,,,,,
0,3110.457272,298103,0,23074.000000,487.151280,585.162268,1120.426230,1786.204575,1194.189929,2917.220131,...,2158.417411,297.268548,4232.251981,1752.312103,22668.641594,405.358406,564.424330,22282.715399,791.284601,"POLYGON ((131474.794 6829694.732, 159329.480 6..."
1,77572.094251,2806808,0,269519.432157,8388.831982,9100.960661,15695.139298,22990.217601,31170.183378,49414.142181,...,27003.753824,4603.010368,32449.703050,24874.099312,260657.622127,8861.810030,11548.586971,261757.964707,7761.467450,"POLYGON ((155043.303 6839542.857, 131474.794 6..."
2,3088.669942,968173,0,88994.682629,2484.927089,2908.125655,4978.403308,7751.069360,9329.671127,16341.426736,...,7589.126120,1107.335031,10467.407311,7686.115760,86680.708189,2313.974439,3263.758292,86510.387615,2484.295013,"POLYGON ((159329.480 6826462.807, 131474.794 6..."
3,17325.273377,947980,0,60477.000000,1450.964793,1651.049158,3245.423295,4815.412723,2760.174677,7936.505467,...,5725.214605,1595.458653,11018.753678,3537.916866,59632.038756,844.961244,1211.265632,59270.721656,1206.278344,"POLYGON ((159328.084 6815907.007, 167458.855 6..."
4,7949.417101,500950,0,65644.000000,1543.339189,1823.892550,3445.670477,5394.022824,3135.642128,8600.938952,...,6522.454270,1271.852634,11719.589911,4011.759084,64954.583417,689.416583,1186.868920,64339.434999,1304.565001,"POLYGON ((153626.902 6638684.970, 101736.000 6..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1307,77431.813814,1787301,0,19222.000000,395.004423,395.259305,619.171437,961.040791,3144.939890,2843.213655,...,1597.952787,197.561834,2412.923655,3000.761874,17603.552477,1618.447523,1666.156810,18112.287523,1109.712477,"POLYGON ((1240063.170 6152594.122, 1202190.139..."
1308,12116.715964,482220,0,27058.000000,814.568046,853.859280,1691.398671,2183.459965,1518.173281,4746.944430,...,3077.288932,301.634764,2828.824291,3043.609867,23270.196179,3787.803821,4021.081326,26869.353797,188.646203,"POLYGON ((1210521.228 6100370.845, 1242409.000..."
1309,26610.896228,1121739,0,79548.000000,2237.150788,2508.597645,4590.135727,6088.399040,4995.500802,14702.826744,...,8532.558864,725.687826,8978.666568,10081.802146,73376.240431,6171.759569,6919.082397,78372.753287,1175.246713,"POLYGON ((1208822.151 6189746.357, 1177994.468..."
